## Setup and Import Libraries

In [2]:
import os
import numpy as np
from sentence_transformers import CrossEncoder
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from utils import load_chroma, word_wrap, project_embeddings

In [4]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(
    filename='data/microsoft_annual_report_2022.pdf',
    collection_name='microsoft_annual_report_2022',
    embedding_function=embedding_function
)
chroma_collection.count()

349

## Re-ranking the Long Tail

In [5]:
query = "What has been the investment in research and development?"

In [6]:
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

In [7]:
for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

• operating expenses increased $ 1. 5 billion or 14 % driven by
investments in gaming, search and news advertising, and windows
marketing. operating expenses research and development ( in millions,
except percentages ) 2022 2021 percentage change research and
development $ 24, 512 $ 20, 716 18 % as a percent of revenue 12 % 12 %
0ppt research and development expenses include payroll, employee
benefits, stock - based compensation expense, and other headcount -
related expenses associated with product development. research and
development expenses also include third - party development and
programming costs, localization costs incurred to translate software
for international markets, and the amortization of purchased software
code and services content. research and development expenses increased
$ 3. 8 billion or 18 % driven by investments in cloud engineering,
gaming, and linkedin. sales and marketing ( in millions, except
percentages ) 2022 2021

competitive in local markets and enable

In [9]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [10]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Scores:
0.8762907
2.644577
-0.2680325
-10.731591
-7.7066016
-5.6469946
-4.2970333
-10.933233
-8.666393
-7.0384297


In [11]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o+1)

New Ordering:
2
1
3
7
6
10
5
9
4
8


## Re-ranking with Query Expansion

In [12]:
original_query = "What were the most important factors that contributed to increases in revenue?"

generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [13]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

In [14]:
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [15]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [16]:
scores = cross_encoder.predict(pairs)

In [17]:
print("Scores:")
for score in scores:
    print(score)

Scores:
-10.119465
-1.1369963
-9.768024
-10.017989
-7.9171767
-3.7948651
-6.90209
-10.042844
-4.3417654
-11.0792675
-3.7681527
-10.187725
-7.503584
-4.651893
-10.000139
-4.8184814
-7.490655
-9.428473
-7.4457927
-8.505107
-5.27475


In [18]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
1
10
5
8
13
15
20
6
18
16
12
4
19
17
2
14
3
7
0
11
9
